In [23]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime as dt
import math
from scipy.stats import norm
import datetime
from abc import ABC, abstractmethod
from dataclasses import dataclass
from enum import Enum
from typing import Union

In [2]:
# def option_category(self):
#         if (self.asset.long_call or 
#             self.asset.long_call_lower or 
#             self.asset.long_call_higher or 
#             self.asset.lower_call or
#             self.asset.higher_call or
#             self.asset.stock):
#             self.assett_class = "Call"
#             self.option_type = "Long"
#         elif (self.asset.short_call or
#             self.asset.short_call_middle):
#             self.assett_class = "Call"
#             self.option_type = "Short"
#         elif (self.asset.long_put or
#              self.asset.lower_put or
#              self.asset.higher_put):
#             self.assett_class = "Put"
#             self.option_type = "Long"
#         elif (self.asset.short_put):
#             self.assett_class = "Put"
#             self.option_type = "Short"
#         else:
#             raise "Non ci sono securities valide inserite"
#         return self

In [3]:
import datetime

class TimeProcessor:
    def __init__(self, expiration_date):
        self.expiration_date = self.parse_expiration_date(expiration_date)

    def parse_expiration_date(self, expiration_date):
        if isinstance(expiration_date, datetime.datetime):
            return expiration_date
        elif isinstance(expiration_date, str):
            for date_format in ['%Y-%m-%d','%d-%m-%Y','%m-%d-%Y', '%m/%d/%Y', '%d/%m/%Y',"%Y-%m-%d"]:
                try:
                    return datetime.datetime.strptime(expiration_date, date_format)
                except ValueError:
                    pass
            raise ValueError("Unsupported date format.")
        else:
            raise TypeError("expiration_date must be a string or a datetime.datetime object.")

    def calculate_time_to_expiration(self):
        today = datetime.datetime.now()
        if self.expiration_date < today:
            return 0  # Return 0 if the expiration date is in the past
        time_to_expiration = (self.expiration_date - today)
        return time_to_expiration

In [4]:
class PresentValueCalculator:
    def __init__(self, risk_free_rate= 0.02):
        """
        Initialize the PresentValueCalculator class.

        Parameters:
        risk_free_rate (float): The risk-free rate as a decimal (e.g., 0.05 for 5%).
        """
        self.risk_free_rate = risk_free_rate

    def calculate_present_value(self, future_value, time_periods):
        """
        Calculate the present value of a future value using the risk-free rate with continuous capitalization.

        Parameters:
        future_value (float): The future value.
        time_periods (float): The number of time periods.

        Returns:
        float: The present value.
        """
        return future_value * math.exp(-self.risk_free_rate * time_periods)
class Assets:
    def __init__(self):
        self.yield_risk_free_returns = True
        self.is_underlying = False
        self.is_derivative = False
        self.is_futures = False
        self.is_option = False
        self.time_to_maturity = None

class Stock(Assets):
    def __init__(self, company_name, ticker_symbol, share_price):
        super().__init__()
        self.company_name = company_name
        self.ticker_symbol = ticker_symbol
        self.share_price = share_price

    def capm(self, Rf, beta):
        return Rf + beta * (E(Rm) - Rf)

class Bonds(Assets):
    def __init__(self, issuer, maturity_date, coupon_rate):
        super().__init__()
        self.issuer = issuer
        self.maturity_date = maturity_date
        self.coupon_rate = coupon_rate

    def duration(self, cash_flows, discount_factors):
        total = 0
        for i, cash_flow in enumerate(cash_flows):
            total += cash_flow * discount_factors[i]
        return total

class Futures(Assets):
    def __init__(self, underlying_asset, expiration_date, contract_size):
        super().__init__()
        self.underlying_asset = underlying_asset
        self.expiration_date = expiration_date
        self.contract_size = contract_size

    def capm(self, Rf, beta):
        return Rf + beta * (E(Rm) - Rf)

In [5]:
# class OptionType:
#     EUROPEAN = "European"
#     AMERICAN = "American"
#     ASIAN = "Asian"

In [6]:
# class OptionStrategy:
#     LONG_CALL = "Long Call"
#     LONG_PUT = "Long Put"
#     SHORT_CALL = "Short Call"
#     SHORT_PUT = "Short Put"
#     COVERED_CALL = "Covered Call"
#     PROTECTIVE_PUT = "Protective Put"
#     BULL_CALL_SPREAD = "Bull Call Spread"
#     BEAR_PUT_SPREAD = "Bear Put Spread"
#     STRADDLE = "Straddle"
#     STRANGLE = "Strangle"
#     IRON_CONDOR = "Iron Condor"
#     BUTTERFLY_SPREAD = "Butterfly Spread"

In [47]:
# class Data:
#     def __init__(self, stock):
#         self.stock = stock

#     def fetch_data(self):
#         df = yf.download(self.stock, period="1y")
#         df = df.sort_values(by="Date")
#         df = df.dropna()
#         df = df.assign(close_day_before=df.Close.shift(1))
#         df['returns'] = ((df.Close - df.close_day_before)/df.close_day_before)
#         sigma = np.sqrt(252) * df['returns'].std()
#         return df

# class OptionData:
#     def __init__(self, stock, strike_price, expiration_date, risk_free_rate, dividend_yield):
#         self.stock = stock
#         self.strike_price = strike_price
#         self.expiration_date = expiration_date
#         self.risk_free_rate = risk_free_rate
#         self.dividend_yield = dividend_yield

#     def fetch_data(self, data):
#         df = data.fetch_data()
#         S = df['Close'].iloc[-1]
#         T = (dt.strptime(self.expiration_date, "%d-%m-%Y") - dt.now()).days / 365
#         return S, T

#     def calculate_option_premium(self, S, T):
#         d1 = (math.log(S / self.strike_price) + (self.risk_free_rate + self.dividend_yield + self.dividend_yield**2 / 2) * T) / (self.dividend_yield * math.sqrt(T))
#         d2 = d1 - self.dividend_yield * math.sqrt(T)

#         if self.strike_price == "Call":
#             return S * norm.cdf(d1) - self.strike_price * math.exp(-self.risk_free_rate * T) * norm.cdf(d2)
#         elif self.strike_price == "Put":
#             return self.strike_price * math.exp(-self.risk_free_rate * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

#     def run(self):
#         data = Data(self.stock)
#         df = data.fetch_data()
#         S, T = self.fetch_data(data)
#         option_premium = self.calculate_option_premium(S, T)
#         print("Option Premium: ", option_premium)

# # Example usage
# stock = "AAPL"
# strike_price = 150
# expiration_date = "2024-06-30"
# risk_free_rate = 0.05
# dividend_yield = 0.02

# calculator = OptionData(stock, strike_price, expiration_date, risk_free_rate, dividend_yield)
# calculator.run()